In [13]:
import fractions
import os
import music21
import pickle
import numpy as np
import pandas as pd
import tensorflow as tf
from keras_preprocessing.text import Tokenizer
from keras_self_attention import SeqSelfAttention
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow import random
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.python.keras.callbacks import ModelCheckpoint


RANDOM_STATE: int = 151

In [3]:
class ChordGeneratorMusicModel:
    def __init__(self, seq_len: int, chords_vocab_size: int):
        random.set_seed(RANDOM_STATE)
        chords_input = Input(shape=(seq_len, 1))
        bilstm = Bidirectional(LSTM(512, return_sequences=True))(chords_input)
        attention = SeqSelfAttention(attention_activation="sigmoid")(bilstm)
        dropout = Dropout(0.3)(attention)
        lstm = LSTM(512, return_sequences=True)(dropout)
        dropout = Dropout(0.3)(lstm)
        pool = GlobalAveragePooling1D()(dropout)
        dense = Dense(256)(pool)
        chord_output = Dense(chords_vocab_size, activation="softmax")(dense)
        self._model = Model(chords_input, chord_output)
        self._model.compile(loss='sparse_categorical_crossentropy', optimizer='rmsprop', metrics=["accuracy"])

    def fit(self, train_x, train_y, **kwargs):
        return self._model.fit(train_x, train_y, **kwargs)

In [4]:
samples: pd.DataFrame = pd.read_csv("./data/beethoven_samples.csv")
samples

,ChordDurationSeq,NextChordDuration
0,rest#4.5 0#1.25 8#0.25 5#4.5 8#1.25 0#0.25 5#1...,rest#0.25
1,0#1.25 8#0.25 5#4.5 8#1.25 0#0.25 5#1.5 8#1.25...,1#0.25
2,8#0.25 5#4.5 8#1.25 0#0.25 5#1.5 8#1.25 0#0.25...,rest#0.25
3,5#4.5 8#1.25 0#0.25 5#1.5 8#1.25 0#0.25 5#1.5 ...,1#0.25
4,8#1.25 0#0.25 5#1.5 8#1.25 0#0.25 5#1.5 5|8#1....,rest#0.25
...,...,...
63777,3|7|10#0.25 3|7|10#0.25 3|7#1/12 7|8|0|3#1/6 7...,10#0.25
63778,3|7|10#0.25 3|7#1/12 7|8|0|3#1/6 7|10#0.25 6|9...,3|7|10#1.0
63779,3|7#1/12 7|8|0|3#1/6 7|10#0.25 6|9#0.25 7|10#0...,rest#0.5
63780,7|8|0|3#1/6 7|10#0.25 6|9#0.25 7|10#0.25 10|3#...,2|5|8|10#1.0


In [6]:
generator_seq_len: int = len(samples["ChordDurationSeq"][0].split(" "))
generator_seq_len

100

In [7]:
X = samples["ChordDurationSeq"]
Y = samples["NextChordDuration"]

tokenizer: Tokenizer = Tokenizer(filters="")
tokenizer.fit_on_texts(X)
tokenizer.fit_on_texts(Y)

X = np.array([z for sublist in tokenizer.texts_to_sequences(X) for z in sublist]).reshape((-1, 100))
Y = np.array([z for sublist in tokenizer.texts_to_sequences(Y) for z in sublist])
scaler: StandardScaler = StandardScaler()
X = scaler.fit_transform(X.reshape(-1, 1)).reshape(-1, generator_seq_len, 1)
train_x, test_x, train_y, test_y = train_test_split(X, Y, shuffle=True, random_state=RANDOM_STATE)

In [9]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
config = tf.config.experimental.set_memory_growth(physical_devices[0], True)
sh_generator: ChordGeneratorMusicModel = ChordGeneratorMusicModel(generator_seq_len, len(tokenizer.word_index) + 1)
checkpoint: ModelCheckpoint = ModelCheckpoint(
    os.path.abspath("./models/shfull-generator-{epoch}-{loss}"),
    period=10,
    monitor='loss',
    verbose=1,
    save_best_only=False,
    mode='min'
)
sh_generator.fit(train_x, train_y, epochs=170, batch_size=64, callbacks=[checkpoint], validation_data=(test_x, test_y))

Epoch 1/170
748/748 [==============================] - 34s 46ms/step - loss: 6.3180 - accuracy: 0.0246 - val_loss: 6.1778 - val_accuracy: 0.0282
Epoch 2/170
748/748 [==============================] - 34s 45ms/step - loss: 6.1054 - accuracy: 0.0293 - val_loss: 6.0994 - val_accuracy: 0.0310
Epoch 3/170
748/748 [==============================] - 34s 45ms/step - loss: 6.0067 - accuracy: 0.0320 - val_loss: 6.0423 - val_accuracy: 0.0315
Epoch 4/170
748/748 [==============================] - 34s 45ms/step - loss: 5.9332 - accuracy: 0.0356 - val_loss: 5.9862 - val_accuracy: 0.0398
Epoch 5/170
748/748 [==============================] - 34s 45ms/step - loss: 5.8479 - accuracy: 0.0423 - val_loss: 5.9345 - val_accuracy: 0.0408
Epoch 6/170
748/748 [==============================] - 34s 45ms/step - loss: 5.7484 - accuracy: 0.0513 - val_loss: 5.8356 - val_accuracy: 0.0514
Epoch 7/170
748/748 [==============================] - 34s 46ms/step - loss: 5.6110 - accuracy: 0.0619 - val_loss: 5.6693 - val_ac

In [14]:
with open("./models/shfull-scaler.pckl", "wb") as f:
    pickle.dump(scaler, f)
with open("./models/shfull-tokenizer.pckl", "wb") as f:
    pickle.dump(tokenizer, f)

In [18]:
class EmbChordGeneratorMusicModel:
    def __init__(self, seq_len: int, chords_vocab_size: int):
        random.set_seed(RANDOM_STATE)
        chords_input = Input(shape=seq_len)
        embedding = Embedding(chords_vocab_size + 1, 100)(chords_input)
        bilstm = Bidirectional(LSTM(512, return_sequences=True))(embedding)
        attention = SeqSelfAttention(attention_activation="sigmoid")(bilstm)
        dropout = Dropout(0.3)(attention)
        lstm = LSTM(1024, return_sequences=True)(dropout)
        dropout = Dropout(0.3)(lstm)
        lstm = LSTM(512, return_sequences=True)(dropout)
        pool = GlobalAveragePooling1D()(lstm)
        dense = Dense(512)(pool)
        chord_output = Dense(chords_vocab_size + 1, activation="softmax")(dense)
        self._model = Model(chords_input, chord_output)
        self._model.compile(loss='sparse_categorical_crossentropy', optimizer='rmsprop', metrics=["accuracy"])

    def fit(self, train_x, train_y, **kwargs):
        return self._model.fit(train_x, train_y, **kwargs)

In [19]:
X = samples["ChordDurationSeq"]
Y = samples["NextChordDuration"]

tokenizer: Tokenizer = Tokenizer(filters="")
tokenizer.fit_on_texts(X)
tokenizer.fit_on_texts(Y)

X = np.array([z for sublist in tokenizer.texts_to_sequences(X) for z in sublist]).reshape((-1, generator_seq_len))
Y = np.array([z for sublist in tokenizer.texts_to_sequences(Y) for z in sublist])
train_x, test_x, train_y, test_y = train_test_split(X, Y, shuffle=True)

In [21]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
config = tf.config.experimental.set_memory_growth(physical_devices[0], True)
train_x, test_x, train_y, test_y = train_test_split(X, Y, shuffle=True)
emb_generator: EmbChordGeneratorMusicModel = EmbChordGeneratorMusicModel(generator_seq_len, len(tokenizer.word_index))
checkpoint: ModelCheckpoint = ModelCheckpoint(
    os.path.abspath("./models/emb-generator-{epoch}-{loss}"),
    period=10,
    monitor='loss',
    verbose=1,
    save_best_only=False,
    mode='min'
)
emb_generator.fit(train_x, train_y, epochs=170, batch_size=64, callbacks=[checkpoint], validation_data=(test_x, test_y))

Epoch 1/170
748/748 [==============================] - 68s 91ms/step - loss: 6.3505 - accuracy: 0.0236 - val_loss: 6.1622 - val_accuracy: 0.0274
Epoch 2/170
748/748 [==============================] - 67s 90ms/step - loss: 6.1141 - accuracy: 0.0308 - val_loss: 6.0669 - val_accuracy: 0.0362
Epoch 3/170
748/748 [==============================] - 68s 90ms/step - loss: 5.9354 - accuracy: 0.0417 - val_loss: 5.9450 - val_accuracy: 0.0494
Epoch 4/170
748/748 [==============================] - 68s 91ms/step - loss: 5.7152 - accuracy: 0.0581 - val_loss: 5.6551 - val_accuracy: 0.0652
Epoch 5/170
748/748 [==============================] - 68s 91ms/step - loss: 5.5044 - accuracy: 0.0741 - val_loss: 5.5071 - val_accuracy: 0.0790
Epoch 6/170
748/748 [==============================] - 68s 91ms/step - loss: 5.3034 - accuracy: 0.0919 - val_loss: 5.4010 - val_accuracy: 0.0863
Epoch 7/170
748/748 [==============================] - 68s 91ms/step - loss: 5.1241 - accuracy: 0.1110 - val_loss: 5.2591 - val_ac

In [29]:
class SimpleEmbChordGeneratorMusicModel:
    def __init__(self, seq_len: int, chords_vocab_size: int):
        random.set_seed(RANDOM_STATE)
        chords_input = Input(shape=seq_len)
        embedding = Embedding(chords_vocab_size + 1, 100)(chords_input)
        bilstm = Bidirectional(LSTM(512, return_sequences=True))(embedding)
        dropout = Dropout(0.3)(bilstm)
        lstm = LSTM(1024, return_sequences=True)(dropout)
        dropout = Dropout(0.3)(lstm)
        lstm = LSTM(512, return_sequences=True)(dropout)
        pool = GlobalAveragePooling1D()(lstm)
        dense = Dense(512)(pool)
        chord_output = Dense(chords_vocab_size + 1, activation="softmax")(dense)
        self._model = Model(chords_input, chord_output)
        self._model.compile(loss='sparse_categorical_crossentropy', optimizer='rmsprop', metrics=["accuracy"])

    def fit(self, train_x, train_y, **kwargs):
        return self._model.fit(train_x, train_y, **kwargs)

In [31]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
config = tf.config.experimental.set_memory_growth(physical_devices[0], True)
simple_emb_generator: SimpleEmbChordGeneratorMusicModel = SimpleEmbChordGeneratorMusicModel(generator_seq_len, len(tokenizer.word_index))
checkpoint: ModelCheckpoint = ModelCheckpoint(
    os.path.abspath("./models/simple-generator-{epoch}-{loss}"),
    period=10,
    monitor='loss',
    verbose=1,
    save_best_only=False,
    mode='min'
)
simple_emb_generator.fit(train_x, train_y, epochs=100, batch_size=64, callbacks=[checkpoint], validation_data=(test_x, test_y))

Epoch 1/100
748/748 [==============================] - 66s 89ms/step - loss: 6.3979 - accuracy: 0.0229 - val_loss: 6.1689 - val_accuracy: 0.0283
Epoch 2/100
748/748 [==============================] - 65s 87ms/step - loss: 6.1497 - accuracy: 0.0281 - val_loss: 6.0418 - val_accuracy: 0.0343
Epoch 3/100
748/748 [==============================] - 65s 87ms/step - loss: 5.9624 - accuracy: 0.0383 - val_loss: 5.8686 - val_accuracy: 0.0439
Epoch 4/100
748/748 [==============================] - 65s 87ms/step - loss: 5.7911 - accuracy: 0.0489 - val_loss: 5.6926 - val_accuracy: 0.0589
Epoch 5/100
748/748 [==============================] - 65s 87ms/step - loss: 5.6152 - accuracy: 0.0680 - val_loss: 5.6389 - val_accuracy: 0.0693
Epoch 6/100
748/748 [==============================] - 65s 87ms/step - loss: 5.4015 - accuracy: 0.0878 - val_loss: 5.4164 - val_accuracy: 0.0965
Epoch 7/100
748/748 [==============================] - 65s 87ms/step - loss: 5.1839 - accuracy: 0.1102 - val_loss: 5.1462 - val_ac

In [ ]:
with open("./models/emp-tokenizer.pckl", "wb") as f:
    pickle.dump(tokenizer, f)